**시간표 관리 프로그램 코드를 클래스를 활용해 객체지향 방식으로 변경하기/0116/20220785/이상권**

In [3]:
import os
import pickle

class Timetable():
    def __init__(self):
        self.weekdays = ['월', '화', '수', '목', '금']
        self.hours = [str(x) for x in range(1, 9)]
        self.classes = {key: {key: ' ' for key, value in dict.fromkeys(self.weekdays).items()} for key in dict.fromkeys(self.hours).keys()}

    def print_TimeTable(self):
        print()
        print('[ TimeTable ]')
        print('-------------')
        print('| |M|T|W|T|F|')
        print('-------------')
        
        for hour in self.classes.keys():
            print('|', end='')
            print(hour, end='|')
            for day in self.classes[hour].keys():
                print(self.classes[hour][day], end='|')
            print()
            
        print('-------------\n')

    def regist(self, day, period, relay, alphabet):
            
        # 요일 입력 오류 시 예외
        if self.weekdays.count(day) == 0:
            raise AbortTransaction('>>> [ERROR] 요일을 잘못 입력하셨습니다!')
        
        # 교시 입력 오류 시 예외
        if int(period) <= 0 or int(period) > 8:
            raise AbortTransaction('>>> [ERROR] 시작 시간을 잘못 입력하셨습니다!')
        
        # 연강 시간 입력 오류 시 예외
        if relay < 1:
            raise AbortTransaction('>>> [ERROR] 연강 시간을 잘못 입력하셨습니다!')
        elif not (1 <= int(period) + relay - 1 <= 8):
            raise AbortTransaction('>>> [ERROR] 등록하려는 수업이 등록 가능 시간을 벗어납니다!') 
        
        # 수업 중복 확인
        if dup_check(day, period, relay, **self.classes):
            raise AbortTransaction('>>> [ERROR] 해당 시간에 이미 수업이 있습니다!')

        for i in range(relay):
            self.classes[str(int(period)+i)][day] = alphabet[0]
            
        self.print_TimeTable()

    def saveTimeTabletxt(self):
        f = open('timetable.txt', 'w')
        
        print()
        print('[ TimeTable ]', file=f)
        print('-------------', file=f)
        print('| |M|T|W|T|F|', file=f)
        print('-------------', file=f)
        
        for hour in self.classes.keys():
            print('|', end='', file=f)
            print(hour, end='|', file=f)
            for day in self.classes[hour].keys():
                print(self.classes[hour][day], end='|', file=f)
            print(file=f)
            
        print('-------------\n', file=f)
        
        f.close()

    def saveTimeTableDic(self):
        with open('timetable.pkl', 'wb') as f:
            pickle.dump(self.classes, f)

    def loadTimeTableTxt(self):
        with open('timetable.txt', 'r') as file:
            for text in file:
                print(text.strip('\n'))

    def loadTimbeTableDic(self):
        if not os.path.isfile('timetable.pkl'):
            raise AbortTransaction('>>> [ERROR] 저장된 시간표가 없습니다!')
        
        with open('timetable.pkl', 'rb') as f:
            mydict = pickle.load(f)
            
            print()
            print('[ TimeTable ]')
            print('-------------')
            print('| |M|T|W|T|F|')
            print('-------------')

            for hour in mydict.keys():
                print('|', end='')
                print(hour, end='|')
                for day in mydict[hour].keys():
                    print(mydict[hour][day], end='|')
                print()

            print('-------------\n')

class AbortTransaction(Exception):
    def __init__(self, msg):
        self.msg = msg

    def __str__(self):
        return self.msg
    
# 중복 확인 함수
def dup_check(day, hour, num, **classdict):  
    dup = False
    
    for i in range(num):
        if classdict[str(int(hour)+i)][day] != ' ':
            dup = True
            break
        
    return dup

def menu(msg, menuList):
    print('[Menu]')
    for i in range(len(menuList)):
        print(f'{i+1}. {menuList[i]}')
    print('0. 종료')
    while True:
        try:
            num = int(input(msg))
            if num == 0:
                break
            elif num > 0 and num <= len(menuList):
                return num
            else:
                print(f'>>> [ERROR] 0 ~ {len(menuList)} 사이의 숫자를 입력하세요.')
        except:
            print('>>> [ERROR] 숫자를 입력하세요.')
            
def main():
    tb = Timetable()
    
    msg = '>>> 원하는 업무를 선택하세요 : '
    main_menu = ['수업 등록', '시간표 저장', '시간표 불러오기']
    save_menu = ['텍스트 저장', '딕셔너리 저장', '둘 다 저장']
    load_menu = ['텍스트 불러오기', '딕셔너리 불러오기']
    
    while True:
        menuSel = menu(msg, main_menu)
        
        if menuSel == 1:
            while True:
                sel = menu(msg, ['수업 등록'])
                
                try:
                    if sel == 1:
                        day, period = input('>>> 등록할 수업의 요일과 교시를 입력하세요 : ').split()
                        relay = int(input('>>> 등록할 수업의 연강 시간을 입력하세요 : '))
                        alphabet = input('>>> 등록할 수업의 알파벳을 입력하세요 : ')

                        tb.regist(day, period, relay, alphabet)

                    else:
                        print('>>> 메인 메뉴로 돌아갑니다.')
                        break
                
                except AbortTransaction as error:
                    print(error)
                    continue
        
        elif menuSel == 2:
            sel = menu(msg, save_menu)
            
            try:
                if sel == 1:
                    tb.saveTimeTabletxt()
                    print('>>> 텍스트로 저장되었습니다.')
                    
                elif sel == 2:
                    tb.saveTimeTableDic()
                    print('>>> 딕셔너리로 저장되었습니다.')
                    
                elif sel == 3:
                    tb.saveTimeTabletxt()
                    tb.saveTimeTableDic()
                    print('>>> 텍스트와 딕셔너리 둘 다 저장되었습니다.')
                    
                else:
                    print('>>> 메인 메뉴로 돌아갑니다.')
                    break
                
            except AbortTransaction as error:
                print(error)
                continue
        
        elif menuSel == 3:
            sel = menu(msg, load_menu)

            try:
                if sel == 1:
                    tb.loadTimeTableTxt()
                    print('>>> 텍스트로 불러왔습니다.')

                elif sel == 2:
                    tb.loadTimbeTableDic()
                    print('>>> 딕셔너리로 불러왔습니다.')

                else:
                    print('>>> 메인 메뉴로 돌아갑니다.')
                    break
                
            except AbortTransaction as error:
                print(error)
                continue
        
        else:
            print('>>> 프로그램을 종료합니다.')
            break
    
if __name__ == '__main__':
    main()

[Menu]
1. 수업 등록
2. 시간표 저장
3. 시간표 불러오기
0. 종료
>>> 원하는 업무를 선택하세요 : 3
[Menu]
1. 텍스트 불러오기
2. 딕셔너리 불러오기
0. 종료
>>> 원하는 업무를 선택하세요 : 1
[ TimeTable ]
-------------
| |M|T|W|T|F|
-------------
|1| | |E| | |
|2| | | | | |
|3|A| | | |D|
|4|A| | | |D|
|5| | | | |D|
|6| | | | | |
|7| | | | | |
|8| | | | | |
-------------

>>> 텍스트로 불러왔습니다.
[Menu]
1. 수업 등록
2. 시간표 저장
3. 시간표 불러오기
0. 종료
>>> 원하는 업무를 선택하세요 : 0
>>> 프로그램을 종료합니다.
